In [54]:
import pandas as pd
import numpy as np
import h5py
import boto3
import tables
import sqlite3
from s3fs.core import S3FileSystem

In [2]:
!python --version

Python 3.7.6


In [3]:
!conda --version

conda 4.8.2


# Million Song Metadata Ingest

Welcome! This notebook is Step 1 in creating the backbone that underlies lucyd. This is functional one-shot code, so please forgive the lack of modularity. 

This notebook is run on an AWS EC2 instance with:
    + Python 3.7.6
    + conda 4.8.2
    + EC2 size = t3.xlarge
    + OS = Amazon Linux AMI, release 2018.03


We'll walk you through the steps we took carefully below, but, in general, the steps are as follows:

    1) Download the data from http://millionsongdataset.com/sites/default/files/AdditionalFiles/msd_summary_file.h5
    2) Download artist level tag database from http://www.ee.columbia.edu/~thierry/artist_term.db
    3) Retrieve all relevant fields from the data
    4) Crosswalk with Spotify URIs
    5) Export to S3

In [55]:
#this .h5 file was retrieved from the link above using this simple command on the EC2 instance:
## wget http://millionsongdataset.com/sites/default/files/AdditionalFiles/msd_summary_file.h5
f = h5py.File('./msd_summary_file.h5', 'r')

<KeysViewHDF5 ['songs']>

## Get Artist-Level Tags to Merge on Later

Using the database we describe below, we're going to make a little dataframe mapping Arist ID to some curated artist level tags. Later, we'll merge these onto the million song metadata by artist_id.

In [57]:
#this .db file was retrieved from the link above using this simple command on the EC2 instance:
## wget http://www.ee.columbia.edu/~thierry/artist_term.db
con = sqlite3.connect("./artist_term.db")
cur = con.cursor()

#query for all MusicBrainz artist-level tags 
##then, make a DF out of it
cur.execute("select * from artist_mbtag;")
artist_mbtags = cur.fetchall()
artist_mbtags = pd.DataFrame(artist_mbtags, columns = ['artist_id','artist_mbtags'])
#the data is unique by artist/tag, so roll it up to exist on a single line delim by comma
artist_mbtags = artist_mbtags.groupby('artist_id')['artist_mbtags'] \
                             .apply(lambda x: ','.join(x)).reset_index().drop_duplicates('artist_mbtags')

#query for all EchoNext artist-level tags
##then, make a DF out of it
cur.execute("select * from artist_term;")
artist_terms = cur.fetchall()
artist_terms = pd.DataFrame(artist_terms, columns = ['artist_id','artist_terms'])
#the data is unique by artist/tag, so roll it up to exist on a single line delim by comma
artist_terms = artist_terms.groupby('artist_id')['artist_terms'] \
                           .apply(lambda x: ','.join(x)).reset_index().drop_duplicates('artist_id')

#close that connection!
con.close()

#slap these two puppies together
terms = pd.merge(artist_terms,artist_mbtags,on = 'artist_id', how = 'outer')
#make both resulting fields the missing string instead of np.NaN
terms['artist_terms'] = np.where(terms['artist_terms'].isnull(),"",terms['artist_terms'])
terms['artist_mbtags'] = np.where(terms['artist_mbtags'].isnull(),"",terms['artist_mbtags'])

#make a new field that's just the concat of those strings
terms['all_terms'] = terms['artist_terms'] + terms['artist_mbtags']
terms = terms.loc[:,['artist_id','all_terms']]
terms.head()

,artist_id,all_terms
0,AR002UA1187B9A637D,"garage rock,country rock,free jazz,oi,space ro..."
1,AR003FB1187B994355,"rock,punk,alternative rock,hip hop,texas,unite..."
2,AR006821187FB5192B,"orchestra,opera,religious music,requiem,califo..."
3,AR009211187B989185,"lovers rock,reggae,roots reggae,uk garage,ball..."
4,AR009SZ1187B9A73F4,"chill-out,future jazz,neofolk,downtempo,folktr..."


## Get Million Song Metadata

We don't need alllll of the data from the Million Song Dataset. Our platform, lucyd, is all about getting the songs you want to hear using only the tags generated by users! Therefore, we're going to intentionally avoid using acoustic characteristics. As a result, we only need some high level information for each song.

For more information about each field, see the field list here: http://millionsongdataset.com/faq/#field-list

In [72]:
#reach in each one and then concat at the end
song_id = pd.Series(f['metadata']['songs']['song_id'])
track_id = pd.Series(f['analysis']['songs']['track_id'])
hotness = pd.Series(f['metadata']['songs']['song_hotttnesss'])
familiar = pd.Series(f['metadata']['songs']['artist_familiarity'])
digital7 = pd.Series(f['metadata']['songs']['track_7digitalid'])
title = pd.Series(f['metadata']['songs']['title'])
artist = pd.Series(f['metadata']['songs']['artist_name'])
artist_id = pd.DataFrame(f['metadata']['songs']['artist_id'])
mode = pd.Series(f['analysis']['songs']['mode'])
key = pd.Series(f['analysis']['songs']['key'])
tempo = pd.Series(f['analysis']['songs']['tempo'])

flat_summary = pd.concat([song_id,track_id,hotness,familiar,digital7,title,artist,artist_id,mode,tempo,key], axis = 1)
flat_summary.columns = ['song_id','track_id','song_hotness','artist_familiarity','7digital_id','title','artist','artist_id','mode','tempo','key']

## Light Cleaning and Crosswalking

All strings are stored as bytestrings. So, we'll decode them to be proper utf-8 encoded. 

After, we'll start crosswalking to pull in the artist level terms and the Spotify URIs.

In [75]:
#iterate through the bytestring columns to decode
for clm in ['song_id','track_id','title','artist']:
    print("Transforming: {}".format(clm))
    flat_summary[clm] = flat_summary[clm].str.decode("utf-8")

#the artist_id field is super pesky, so we had to do this one separate for some reason
flat_summary['temp'] = flat_summary['artist_id'].str.decode("utf-8") 
flat_summary.drop('artist_id', axis = 1, inplace = True)
flat_summary.rename(columns = {'temp':'artist_id'}, inplace = True)
flat_summary

Transforming: song_id
Transforming: track_id
Transforming: title
Transforming: artist


,song_id,track_id,song_hotness,artist_familiarity,7digital_id,title,artist,mode,tempo,key,artist_id
0,SOQMMHC12AB0180CB8,TRMMMYQ128F932D901,0.542899,0.649822,7032331,Silent Night,Faster Pussy cat,0,87.002,10,ARYZTJS1187B98C555
1,SOVFVAK12A8C1350D9,TRMMMKD128F425225D,0.299877,0.439604,1514808,Tanssi vaan,Karkkiautomaatti,1,150.778,9,ARMVN3U1187FB3A1EB
2,SOGTUKN12AB017F4F1,TRMMMRX128F93187D9,0.617871,0.643681,6945353,No One Could Ever,Hudson Mohawke,1,177.768,7,ARGEKB01187FB50750
3,SOBNYVR12A8C13558C,TRMMMCH128F425532C,NaN,0.448501,2168257,Si Vos Querés,Yerba Brava,1,87.433,7,ARNWYLR1187B9B2F9C
4,SOHSBXH12A8C13B0DF,TRMMMWA128F426B589,NaN,0.000000,2264873,Tangle Of Aspens,Der Mystic,0,140.035,5,AREQDTE1269FB37231
...,...,...,...,...,...,...,...,...,...,...,...
999995,SOTXAME12AB018F136,TRYYYUS12903CD2DF0,NaN,0.528617,7522478,O Samba Da Vida,Kiko Navarro,0,92.159,7,AR7Z4J81187FB3FC59
999996,SOXQYIQ12A8C137FBB,TRYYYJO128F426DA37,NaN,0.401500,1632096,Jago Chhadeo,Kuldeep Manak,1,156.132,5,ART5FZD1187B9A7FCF
999997,SOHODZI12A8C137BB3,TRYYYMG128F4260ECA,NaN,0.556918,2219291,Novemba,Gabriel Le Mar,0,137.089,11,ARZ3R6M1187B9AF750
999998,SOLXGOR12A81C21EB7,TRYYYDJ128F9310A21,NaN,0.403668,5472456,Faraday,Elude,0,137.928,6,ARCMCOK1187B9B1073


In [56]:
#retrieving the stored musicbrainz-to-spotify crosswalk
##this was created in a different notebook
bucket='sagemaker-msdsubset'
data_key = 'musicbrainz_spotify_id_crosswalk.csv'
ACCESS_KEY = 'ENTER YOUR ACCESS KEY HERE'
SECRET_KEY = 'ENTER YOUR SECRET KEY HERE'
s3 = S3FileSystem(key=ACCESS_KEY, secret=SECRET_KEY)
with s3.open('sagemaker-msdsubset/songid_spotifyuri_crosswalk.csv') as xwalk:
    spotify_xwalk = pd.read_csv(xwalk)
    
#they were created such that a non-match was called "no_match"
##let's just turn that into the empty string
spotify_xwalk = spotify_xwalk.loc[:,['song_id','spotify_uri_final']]
spotify_xwalk['spotify_uri_final'] = np.where(spotify_xwalk['spotify_uri_final'] == 'no_match',""
                                              ,spotify_xwalk['spotify_uri_final'])
spotify_xwalk.head()

,song_id,spotify_uri_final
0,SOQMMHC12AB0180CB8,
1,SOVFVAK12A8C1350D9,spotify:track:6DOmOjeTc3btomrfFfPgy8
2,SOGTUKN12AB017F4F1,spotify:track:41RpZW2lxAdnqDd2nMBzLQ
3,SOBNYVR12A8C13558C,spotify:track:7z4BZV7eZO1bqVKwAeTmou
4,SOHSBXH12A8C13B0DF,spotify:track:2poHURuOfVNbzZdivAwtOH


In [76]:
#we found some more Spotify URIs laying around here: https://archive.org/details/thisismyjam-datadump

#mapping:
##wget http://millionsongdataset.com/sites/default/files/thisismyjam/jam_to_msd.tsv
jam_walk = pd.read_csv(r'./myjam/jam_to_msd.tsv', "\t", names = ['jam_id', 'track_id'])

#uris (you'd have to also unzip):
##wget https://archive.org/download/thisismyjam-datadump/thisismyjam-datadump.zip
jam = pd.read_csv(r"./myjam/archive/jams.tsv", "\t", error_bad_lines = False)

#get rid of null uris
jam = jam[~jam['spotify_uri'].isnull()]

#slap on the MSD ID
jam = pd.merge(jam,jam_walk,on = 'jam_id')
jam.drop_duplicates("track_id", inplace = True)
jam = jam.loc[:,['track_id','spotify_uri']]

b'Skipping line 7872: expected 7 fields, saw 8\nSkipping line 11730: expected 7 fields, saw 9\nSkipping line 14131: expected 7 fields, saw 8\nSkipping line 58054: expected 7 fields, saw 8\nSkipping line 58754: expected 7 fields, saw 8\n'
b'Skipping line 847129: expected 7 fields, saw 8\n'
b'Skipping line 1091153: expected 7 fields, saw 8\nSkipping line 1175375: expected 7 fields, saw 8\n'
b'Skipping line 1225935: expected 7 fields, saw 8\nSkipping line 1255357: expected 7 fields, saw 8\nSkipping line 1279671: expected 7 fields, saw 8\n'
b'Skipping line 1330675: expected 7 fields, saw 8\n'
b'Skipping line 1448033: expected 7 fields, saw 8\nSkipping line 1543893: expected 7 fields, saw 8\n'
b'Skipping line 1579569: expected 7 fields, saw 8\nSkipping line 1612448: expected 7 fields, saw 8\n'
b'Skipping line 1784588: expected 7 fields, saw 8\n'


In [77]:
#add the artist temrs from above
flat_summary = pd.merge(flat_summary, terms, on = 'artist_id', how = 'left')
#add the scraped URIs
flat_summary = pd.merge(flat_summary, spotify_xwalk, on = 'song_id', how = 'left')
#add the jams URIs
flat_summary = pd.merge(flat_summary, jam, on = 'track_id', how = 'left')
#make URI the missing string if its null
flat_summary['spotify_uri_final'] = np.where(flat_summary['spotify_uri_final'] == '',
                                            flat_summary['spotify_uri'],
                                            flat_summary['spotify_uri_final'])
#get rid of the interim field
del flat_summary['spotify_uri']
flat_summary.head()

In [80]:
#and upload to S3
flat_summary.to_csv(r'./flat_summary.csv', index = False, header = False)
ACCESS_KEY = 'ENTER YOUR ACCESS KEY HERE'
SECRET_KEY = 'ENTER YOUR SECRET KEY HERE'
s3 = boto3.Session(aws_access_key_id=ACCESS_KEY,aws_secret_access_key=SECRET_KEY).resource('s3')
bucket = s3.Bucket('sagemaker-msdsubset')
bucket.upload_file(r'./flat_summary.csv', Key = 'flat_summary_04_09_20.csv')